# INFO 2950 Data Cleaning Appendix 

## __Fund Investment Decisions: Mutual Funds or Exchange Traded Funds?__

By Cameron Robinson (ctr48) , David Vilensky (dlv46) , Cassandra Kaufman (cpk57) , and Emily Sobel (es983)

---------------------------------------------------------------------------------------------------------------------------------------------------------------------

# ***__Please Read!!!__ ***

__The dataframes obtained at the end of this data cleaning appendix **ARE NOT** the final dataframes we submit. Although they are very similar, the final dataframes that we actually submit are created later on in our exploratory analysis, where we create and append several of our own predictor variables to the dataframe. The reason we could not work with one cleaned DataFrame is because a lot of our analyses involved dropping and transforming different columns in the dataset, which messed up correlation values for unrelated analyses, which we will describe in greater detail in our Data Limitations section. To avoid this unintended consequence, we split our data cleaning up into two sections. We will outline our thought process below. The preliminary analysis prepares the Treynor Ratio columns for analysis and saves the lightly cleaned dataframes for future use. The advanced cleaning section utilizes the cleanbykeyword method to create a master dataframe for each type of fund, as well as separate dataframes containing related information (Examples: return history, sector distribution).The point at which we consolidate all of our smaller dataframes into final dataframes is explicity noted in our Final Report.__

## Importing

In [1]:
# imports and settings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import chi2_contingency
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import re

%matplotlib inline
plt.rcParams['figure.figsize'] = (12.0, 8.0)
plt.style.use('seaborn-poster')

## Data Overview



The file contains 24,821 Mutual Funds and 1,680 ETFs with general aspects (as Total Net Assets, management company and size), portfolio indicators (as cash, stocks, bonds, and sectors), returns (as yeartodate, 2020-11) and financial ratios (as price/earning, Treynor and Sharpe ratios, alpha, and beta). This data was scraped from Yahoo Finance by kaggle user Stefano Leone.

Link to dataset: https://www.kaggle.com/stefanoleone992/mutual-funds-and-etfs


__Exchange Traded Funds Dataset: 'ETFs.csv'__


In [2]:
df_etf=pd.read_csv('USData/ETFs.csv')

__Mutual Funds Dataset: 'Mutual Funds.csv'__

In [3]:
df_mf=pd.read_csv('USData/Mutual Funds.csv')

## Data Description

The two datasets used were created to allow for multiple comparisons regarding portfolio decisions from investment managers in Mutual Funds and portfolio restrictions to the indexes in Exchnage traded funds. Each of the datasets are composed of publicly available information and were not funded by any companies or private organizations. The instances in each of the datasets are  funds with the respective fund type, either Exchange traded funds in the ETF.csv or Mutual funds in the Mutual Funds.csv. The columns in each of the datasets contain information on the respective fund's financial ratios, sector investments, risk ratios, and returns. The data in each of the datasets was collected over a time period between 2011 and 2020. The original datasets don't have any errors but they do contain null information in the form of empty columns that were dropped during our data cleaning. This preprocessing and all other preprocessing of the data found in the two datasets is extensively documented in our data cleaning section and we have made no additional modifications or alterations to the data. The creator of the datasets placed no copyrights or restrictions on their use and under its license of CC0: Public Domain, we have permission to access and use its both of the datsets content for our project. 
 

## __Preliminary Data Cleaning__

__The first step of the preliminary data cleaning is to drop all of the null columns and all of the unused columns from our DataFrames__

In [4]:
empty_cols2 = [col for col in df_etf.columns if df_etf[col].isnull().all()]
df_etf.drop(empty_cols2,axis=1,inplace=True)

throwaway_cols=[
    'quarters_up',
    'quarters_down',
    'top10_holdings',
    'years_up',
    'years_down',
    'currency'
]
df_mf.drop(columns=throwaway_cols,axis=1,inplace=True)
df_etf.drop(columns=throwaway_cols[2:],axis=1,inplace=True)


__The second step of the Preliminary Data Cleaning section is creating a cleanbykeyword method. Since our datasets are rather large, we created a method that allowed us to break down and sort either dataset based off keywords in column names defined below__

In [5]:
### Creates and returns a subset of df by keeping only the columns with 'keyword' in their column name. 
def cleanbykeyword(df,keyword):
    """
    Searches and creates a list of all of the column names in df that contain the keyword.
    Subsets the df according to this list, then creates a df_new dataframe with this 
    information and drops the corresponding columns from the original df.

    Arguments:
        df: Dataset pandas DataFrame.
        cols: List of columns to be included in calculations. 
        keyword: Word to be removed from column name in order to preserve clarity with labeling.
        
        
    Return: dataframe containing data matching keyword.
    """
    col_list=[ i for i in df.columns if (keyword in i)] 
    #print(col_list)
    df_new=df[col_list]
    df.drop(columns=col_list,axis=1,inplace=True) #removes duplicate data from df_mf masterframe
    df_new.insert(0,'fund_symbol',df.fund_symbol) #inserts the fund symbols as the 1st column of the new data set
    return (df_new)

__The final step is to get rid of the category data. While we do make use of some categorical information, approximately half of the columns contain categorical data that is either missing or irrelevant to our analyses so it is much easier to remove all of the category data and add back the few columns that we end up using rather than clogging up our dataframes with unused or misssing information__

In [6]:
df_mf_category_data=cleanbykeyword(df_mf,'category_')
df_etf_category_data=cleanbykeyword(df_etf,'category_')

#Adds additional category data to our df
df_mf['category_return_ytd']=df_mf_category_data['category_return_ytd']
df_etf['category_return_ytd']=df_etf_category_data['category_return_ytd']

df_mf['category_return_3years']=df_mf_category_data['category_return_3years']
df_etf['category_return_3years']=df_etf_category_data['category_return_3years']

df_mf['category_return_5years']=df_mf_category_data['category_return_5years']
df_etf['category_return_5years']=df_etf_category_data['category_return_5years']

df_mf['category_return_10years']=df_mf_category_data['category_return_10years']
df_etf['category_return_10years']=df_etf_category_data['category_return_10years']

In [7]:
df_etf.head()

,fund_symbol,fund_extended_name,fund_family,inception_date,category,investment_strategy,investment_type,size_type,fund_net_annual_expense_ratio,asset_stocks,...,fund_sharpe_ratio_3years,fund_sharpe_ratio_5years,fund_sharpe_ratio_10years,fund_treynor_ratio_3years,fund_treynor_ratio_5years,fund_treynor_ratio_10years,category_return_ytd,category_return_3years,category_return_5years,category_return_10years
0,1305,Daiwa ETF TOPIX,Daiwa Asset Management Co Ltd,2001-07-11,NaN,The investment seeks to track the price and yi...,Blend,Large,0.00,100.00,...,0.34,0.59,0.60,3.92,8.01,9.29,NaN,NaN,NaN,NaN
1,1306,Nomura TOPIX ETF,Nomura Asset Management Co Ltd,2001-07-11,NaN,The investment seeks to track the price and yi...,Blend,Large,0.05,99.99,...,0.34,0.59,0.61,3.94,8.03,9.33,NaN,NaN,NaN,NaN
2,1308,Nikko Exchange Traded Index TOPIX,Nikko Asset Management Co Ltd,2001-12-20,NaN,The investment seeks to track the price and yi...,Blend,Large,0.00,99.08,...,0.34,0.59,0.60,3.93,8.03,9.31,NaN,NaN,NaN,NaN
3,1310,Daiwa ETF TOPIX Core 30,Daiwa Asset Management Co Ltd,2002-03-28,NaN,The investment seeks to track the price and yi...,Blend,Large,0.00,100.00,...,0.14,0.34,0.44,1.00,4.08,6.18,NaN,NaN,NaN,NaN
4,1311,Nomura TOPIX Core 30 ETF,Nomura Asset Management Co Ltd,2002-04-02,NaN,The investment seeks to track the price and yi...,Blend,Large,0.00,99.90,...,0.14,0.34,0.44,1.02,4.10,6.24,NaN,NaN,NaN,NaN


In [8]:
df_mf.head()

,fund_symbol,fund_extended_name,fund_family,inception_date,category,rating,return_rating,risk_rating,investment_strategy,investment_type,...,fund_sharpe_ratio_3years,fund_sharpe_ratio_5years,fund_sharpe_ratio_10years,fund_treynor_ratio_3years,fund_treynor_ratio_5years,fund_treynor_ratio_10years,category_return_ytd,category_return_3years,category_return_5years,category_return_10years
0,AAAAX,DWS RREEF Real Assets Fund - Class A,DWS,2007-07-29,World Allocation,3.0,3.0,3.0,The investment seeks total return in excess of...,Blend,...,0.18,0.33,0.24,1.32,2.77,1.91,-5.47,1.10,3.97,4.61
1,AAADX,Aberdeen Income Builder Fund Class A,Aberdeen,2011-12-29,Allocation - 50% to 70% Equity,4.0,5.0,5.0,The investment seeks income; long-term growth ...,Blend,...,0.93,0.74,0.89,6.82,5.38,7.1,0.07,4.90,6.23,7.13
2,AAAGX,Thrivent Large Cap Growth Fund Class A,Thrivent Funds,1999-10-28,Large Growth,3.0,3.0,4.0,The investment seeks long-term capital appreci...,Growth,...,0.92,0.88,0.93,16.87,13.48,13.03,17.54,15.98,14.64,14.19
3,AAAIX,American Century Strategic Allocation: Aggress...,American Century Investments,2000-07-31,Allocation - 70% to 85% Equity,4.0,4.0,3.0,The investment seeks the highest level of tota...,Blend,...,0.41,0.61,0.73,3.7,5.05,5.87,-2.67,3.98,6.21,7.46
4,AAANX,Horizon Active Asset Allocation Fund Investor ...,Horizon Investments,2012-01-30,Tactical Allocation,3.0,4.0,5.0,The investment seeks capital appreciation. The...,Blend,...,0.17,0.37,NaN,0.86,2.73,NaN,-1.74,2.24,4.36,4.26


## __Advanced Data Cleaning__

__The first step of our Advanced Data Cleaning section is to prepare the Treynor Ratio columns for analysis and saves the lightly cleaned dataframes for future use. In the initial Kaggle dataset, the Treynor Ratios were recorded as strings so we created a str_to_float( ) method which converts the specified column from str to float.__

In [9]:
def str_to_float(df,column):
    '''
    This method converts the values of a specific column in a given dataframe from strings to floats\n",
    '''
    list=[]
    col_id=df.columns.get_loc(column)
    for i in range(df.shape[0]):
        treynor=df.iloc[i,col_id]
        if type(treynor) == None:
            list.append(-999)
        if type(treynor) != None:
            if type(treynor) == str:
                treynor = treynor.replace(',','')
            f=float(treynor)
            list.append(f)
    return(list)


   


__The second step was to populate the Treynor Columns with the corresponding column mean. The way we wrote the str_to_float method assigns a value of -999 to every None-type in the specified column. So, we create a follow-up method called
fill_with_mean() that calculates the mean of all values above -990 and then replaces any values below -990 with the calculated mean.__

In [10]:
#Fills none types with the average value
#I designed the str_to_float method to assign -100 for missing Treynor ratios so they could be exclded from calculations. This way we avoid dropping the 
def fill_with_mean(df,column):
    '''
    Use:
    This method is an addendum to the str_to_float method. A lot of the 
    Fills the specified column in df with 
    '''
    col_id=df.columns.get_loc(column)
    mean=df[df[column]>-990][column].mean()
    for i in range(df.shape[0]):
        treynor=df.iloc[i,col_id]
        if treynor < -990:
            df.iloc[i,col_id]=mean           
    return

In [11]:
#converting treynor columns from strings to floats ETF 
df_mf['fund_treynor_ratio_3years'] = str_to_float(df_mf,'fund_treynor_ratio_3years')
fill_with_mean(df_mf,'fund_treynor_ratio_3years')
df_mf['fund_treynor_ratio_5years'] = str_to_float(df_mf,'fund_treynor_ratio_5years')
fill_with_mean(df_mf,'fund_treynor_ratio_5years')
df_mf['fund_treynor_ratio_10years'] = str_to_float(df_mf,'fund_treynor_ratio_10years')
fill_with_mean(df_mf,'fund_treynor_ratio_10years')

#converting treynor columns from strings to floats MUTUAL FUNDS  
df_etf['fund_treynor_ratio_3years'] = str_to_float(df_etf,'fund_treynor_ratio_3years')
fill_with_mean(df_etf,'fund_treynor_ratio_3years')
df_etf['fund_treynor_ratio_5years'] = str_to_float(df_etf,'fund_treynor_ratio_5years')
fill_with_mean(df_etf,'fund_treynor_ratio_5years')
df_etf['fund_treynor_ratio_10years'] = str_to_float(df_etf,'fund_treynor_ratio_10years')
fill_with_mean(df_etf,'fund_treynor_ratio_10years')


 __Exports prelminary datasets to be used in later analyses__


In [12]:
df_mf.to_csv('preliminaryMF.csv',index=False)
df_etf.to_csv('preliminaryETF.csv',index=False)


In [15]:
df_etf.fund_treynor_ratio_5years

0       8.01
1       8.03
2       8.03
3       4.08
4       4.10
        ... 
1675   -4.46
1676    0.00
1677    5.56
1678   -8.72
1679    1.92
Name: fund_treynor_ratio_5years, Length: 1680, dtype: float64

__The last step of our Advanced Data Cleanign Section is to implement our clean by keyword method to break the dataframe into more easily accessible parts:__

In [13]:
#Cleaning by fund_return
df_mf_return_history=cleanbykeyword(df_mf,'fund_return_')
df_etf_return_history=cleanbykeyword(df_etf,'fund_return_')

#Adds fund_return_ytd column to main datafrme and removes from return_history subframe
df_mf['fund_return_ytd']=df_mf_return_history['fund_return_ytd']
df_etf['fund_return_ytd']=df_etf_return_history['fund_return_ytd']
df_mf_return_history=df_mf_return_history.drop(columns=['fund_return_ytd'])
df_etf_return_history=df_etf_return_history.drop(columns=['fund_return_ytd'])


df_mf_return_history.to_csv('mf_returns.csv',index=False)
df_etf_return_history.to_csv('etf_returns.csv',index=False)

df_mf_return_history.head()

,fund_symbol,fund_return_1month,fund_return_3months,fund_return_1year,fund_return_3years,fund_return_5years,fund_return_10years,fund_return_2019,fund_return_2018,fund_return_2017,...,fund_return_2012_q2,fund_return_2012_q1,fund_return_2011_q4,fund_return_2011_q3,fund_return_2011_q2,fund_return_2011_q1,fund_return_2010_q4,fund_return_2010_q3,fund_return_2010_q2,fund_return_2010_q1
0,AAAAX,-1.71,-1.02,-5.22,3.03,4.08,2.38,21.43,-5.37,14.67,...,-0.36,4.22,3.28,-8.39,-0.38,2.44,3.73,8.55,-2.53,2.23
1,AAADX,0.30,4.70,15.76,9.71,8.46,10.88,21.48,-5.53,17.05,...,-4.72,10.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAAGX,-4.57,1.48,32.91,19.44,15.26,14.71,31.61,1.48,27.70,...,-5.23,16.35,9.91,-17.18,-0.95,3.93,11.38,10.39,-14.29,4.09
3,AAAIX,-1.32,1.22,8.36,6.79,8.19,8.66,24.67,-7.71,19.31,...,-3.20,11.44,8.40,-13.94,0.89,4.49,8.73,11.37,-8.13,3.92
4,AAANX,-2.43,0.56,1.73,3.04,5.73,NaN,20.60,-8.79,22.25,...,3.27,-2.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df_etf_return_history.head()

,fund_symbol,fund_return_1month,fund_return_3months,fund_return_1year,fund_return_3years,fund_return_5years,fund_return_10years,fund_return_2019,fund_return_2018,fund_return_2017,fund_return_2016,fund_return_2015,fund_return_2014,fund_return_2013,fund_return_2012,fund_return_2011,fund_return_2010
0,1305,1.22,-3.11,-5.17,3.83,7.86,9.25,NaN,NaN,22.06,0.17,11.90,10.12,54.10,20.69,-17.07,0.84
1,1306,1.30,-3.10,-5.21,3.88,7.87,9.27,NaN,NaN,22.06,0.21,11.91,10.13,54.12,20.64,-17.04,0.93
2,1308,1.31,-3.08,-5.09,3.85,7.87,9.22,NaN,NaN,22.09,0.18,11.92,10.13,54.19,20.70,-17.06,0.87
3,1310,0.27,-3.79,-6.80,0.95,4.44,6.70,NaN,NaN,14.83,-2.51,9.12,4.12,54.44,27.50,-23.37,-2.15
4,1311,0.13,-3.48,-6.94,0.99,4.45,6.70,NaN,NaN,14.84,-2.49,9.09,4.17,54.80,27.60,-23.27,-2.00


In [15]:
#Cleaning by ratios
df_mf_ratios=cleanbykeyword(df_mf,'_ratio')
df_etf_ratios=cleanbykeyword(df_etf,'_ratio')

df_mf_ratios.to_csv('mf_ratios.csv',index=False)
df_etf_ratios.to_csv('etf_ratios.csv',index=False)

df_mf_ratios.head()

,fund_symbol,fund_net_annual_expense_ratio,price_earnings_ratio,price_book_ratio,price_sales_ratio,price_cashflow_ratio,fund_sharpe_ratio_3years,fund_sharpe_ratio_5years,fund_sharpe_ratio_10years,fund_treynor_ratio_3years,fund_treynor_ratio_5years,fund_treynor_ratio_10years
0,AAAAX,1.22,19.41,1.83,2.30,11.13,0.18,0.33,0.24,1.32,2.77,1.91
1,AAADX,1.43,21.21,3.60,3.26,15.29,0.93,0.74,0.89,6.82,5.38,7.10
2,AAAGX,1.12,38.70,11.46,5.31,26.26,0.92,0.88,0.93,16.87,13.48,13.03
3,AAAIX,0.63,21.81,2.45,1.44,11.45,0.41,0.61,0.73,3.70,5.05,5.87
4,AAANX,1.45,23.56,2.53,1.58,12.25,0.17,0.37,NaN,0.86,2.73,NaN


In [16]:
#Cleaning by sector
df_mf_sectors=cleanbykeyword(df_mf,'sector')
df_etf_sectors=cleanbykeyword(df_etf,'sector')

df_mf_sectors.to_csv('mf_sectors.csv',index=False)
df_etf_sectors.to_csv('etf_sectors.csv',index=False)
df_etf_sectors.head()


,fund_symbol,sector_basic_materials,sector_consumer_cyclical,sector_financial_services,sector_real_estate,sector_consumer_defensive,sector_healthcare,sector_utilities,sector_communication_services,sector_energy,sector_industrials,sector_technology
0,1305,7.81,17.26,11.73,3.19,9.44,7.05,1.96,4.95,1.7,20.29,14.61
1,1306,7.81,17.26,11.73,3.19,9.43,7.06,1.96,4.95,1.7,20.29,14.62
2,1308,7.74,17.33,11.57,3.22,9.53,7.07,1.99,4.98,1.7,20.23,14.62
3,1310,2.41,14.25,16.26,1.74,7.93,4.55,0.00,15.52,2.0,16.26,19.06
4,1311,2.44,14.26,16.25,1.74,7.91,4.56,0.00,15.52,2.0,16.32,18.99


In [17]:
#assets
df_mf_assets=cleanbykeyword(df_mf,'asset')
df_etf_assets=cleanbykeyword(df_etf,'asset')
df_mf_assets.to_csv('mf_assets.csv',index=False)
df_etf_assets.to_csv('etf_assets.csv',index=False)
df_mf_assets.head()

,fund_symbol,asset_cash,asset_stocks,asset_bonds,asset_others,asset_preferred,asset_convertable,net_asset_value
0,AAAAX,1.88,63.87,15.31,18.95,0.00,0.00,6.979100e+08
1,AAADX,3.49,48.14,48.37,0.00,0.00,0.00,8.938000e+07
2,AAAGX,1.54,98.46,0.00,0.00,0.00,0.00,1.490000e+09
3,AAAIX,2.75,78.52,18.48,0.01,0.07,0.14,7.072500e+08
4,AAANX,1.54,98.45,0.00,0.01,0.00,0.00,4.955100e+08


__For the last 4 keywords we will not be saving to csv files as it is unessecary.__

In [18]:
#Cleaning by standard deviation
df_mf_standard_deviations= cleanbykeyword(df_mf,'standard_deviation')
df_etf_standard_deviations=cleanbykeyword(df_etf,'standard_deviation')
df_etf_standard_deviations.head()

#Cleaning by Beta
df_mf_beta=cleanbykeyword(df_mf,'beta')
df_etf_beta=cleanbykeyword(df_etf,'beta')
df_etf_beta.head()

#credit
df_mf_credit=cleanbykeyword(df_mf,'credit')
df_etf_credit=cleanbykeyword(df_etf,'credit')
df_mf_credit.head()

#R squared
df_mf_rsquare=cleanbykeyword(df_mf,'squared')
df_etf_rsquare=cleanbykeyword(df_etf,'squared')
df_etf_rsquare.head()

,fund_symbol,fund_r_squared_3years,fund_r_squared_5years,fund_r_squared_10years
0,1305,99.72,99.70,99.68
1,1306,99.72,99.69,99.68
2,1308,99.72,99.69,99.68
3,1310,96.51,97.19,96.63
4,1311,96.52,97.20,96.64


## Looking at our Final DataFrames

In [19]:
df_mf.to_csv('df_mf.csv',index=False)
df_mf.head()

,fund_symbol,fund_extended_name,fund_family,inception_date,category,rating,return_rating,risk_rating,investment_strategy,investment_type,...,fund_alpha_5years,fund_alpha_10years,fund_mean_annual_return_3years,fund_mean_annual_return_5years,fund_mean_annual_return_10years,category_return_ytd,category_return_3years,category_return_5years,category_return_10years,fund_return_ytd
0,AAAAX,DWS RREEF Real Assets Fund - Class A,DWS,2007-07-29,World Allocation,3.0,3.0,3.0,The investment seeks total return in excess of...,Blend,...,-2.86,-3.86,0.31,0.38,0.23,-5.47,1.10,3.97,4.61,-7.60
1,AAADX,Aberdeen Income Builder Fund Class A,Aberdeen,2011-12-29,Allocation - 50% to 70% Equity,4.0,5.0,5.0,The investment seeks income; long-term growth ...,Blend,...,-0.30,-0.15,0.80,0.72,0.92,0.07,4.90,6.23,7.13,0.30
2,AAAGX,Thrivent Large Cap Growth Fund Class A,Thrivent Funds,1999-10-28,Large Growth,3.0,3.0,4.0,The investment seeks long-term capital appreci...,Growth,...,2.92,0.77,1.65,1.30,1.25,17.54,15.98,14.64,14.19,24.20
3,AAAIX,American Century Strategic Allocation: Aggress...,American Century Investments,2000-07-31,Allocation - 70% to 85% Equity,4.0,4.0,3.0,The investment seeks the highest level of tota...,Blend,...,-0.78,-0.48,0.64,0.72,0.75,-2.67,3.98,6.21,7.46,3.17
4,AAANX,Horizon Active Asset Allocation Fund Investor ...,Horizon Investments,2012-01-30,Tactical Allocation,3.0,4.0,5.0,The investment seeks capital appreciation. The...,Blend,...,-4.38,NaN,0.39,0.56,NaN,-1.74,2.24,4.36,4.26,-3.26


In [20]:
df_etf.to_csv('df_etf.csv',index=False)
df_etf.head()

,fund_symbol,fund_extended_name,fund_family,inception_date,category,investment_strategy,investment_type,size_type,fund_yield,fund_alpha_3years,fund_alpha_5years,fund_alpha_10years,fund_mean_annual_return_3years,fund_mean_annual_return_5years,fund_mean_annual_return_10years,category_return_ytd,category_return_3years,category_return_5years,category_return_10years,fund_return_ytd
0,1305,Daiwa ETF TOPIX,Daiwa Asset Management Co Ltd,2001-07-11,NaN,The investment seeks to track the price and yi...,Blend,Large,1.84,0.29,0.39,0.35,0.40,0.72,0.86,NaN,NaN,NaN,NaN,-6.62
1,1306,Nomura TOPIX ETF,Nomura Asset Management Co Ltd,2001-07-11,NaN,The investment seeks to track the price and yi...,Blend,Large,1.77,0.31,0.40,0.38,0.41,0.72,0.86,NaN,NaN,NaN,NaN,-6.43
2,1308,Nikko Exchange Traded Index TOPIX,Nikko Asset Management Co Ltd,2001-12-20,NaN,The investment seeks to track the price and yi...,Blend,Large,1.84,0.30,0.41,0.36,0.40,0.72,0.86,NaN,NaN,NaN,NaN,-6.47
3,1310,Daiwa ETF TOPIX Core 30,Daiwa Asset Management Co Ltd,2002-03-28,NaN,The investment seeks to track the price and yi...,Blend,Large,2.57,-2.58,-3.41,-2.56,0.18,0.46,0.68,NaN,NaN,NaN,NaN,-6.69
4,1311,Nomura TOPIX Core 30 ETF,Nomura Asset Management Co Ltd,2002-04-02,NaN,The investment seeks to track the price and yi...,Blend,Large,1.82,-2.56,-3.39,-2.50,0.18,0.47,0.68,NaN,NaN,NaN,NaN,-6.94


In [22]:
df_mf.fund_treynor_ratio_3years

AttributeError: 'DataFrame' object has no attribute 'fund_treynor_ratio_3years'